# Algorithms in Python

**Welcome!** This notebook will teach you about the concept of algorithm in Python. By the end of this notebook, you'll exprience writing your own algorithm to solve a problem, you will know the concept of computational complexity, and also learn several types of typical problems in computer programming. More over, you will see how different problems are handled by different external libraries.

<hr>

## The definition

An algorithm is a finite sequence of rigorous instructions

### Finding the minimum

We have a list of numbers, how do we find which is the minimum value?

In [ ]:
import random

numbers = [random.random() * 1000 for _ in range(1000000)]
numbers

To find the minimum value, we can design the following steps:

- we assume the first item is the minimum. and record its index (which is 0).
- we compare the recorded minimum value with the remaining items, one by one.
- if we see a smaller value, we update our recorded index.

In [ ]:
i_min = 0   # index of the minimum

for i in range(1, len(numbers)):
    if numbers[i] < numbers[i_min]:
        i_min = i

print("The", i_min, "-th item has the minimum value.")
print("The minimum value is", numbers[i_min])

### Finding the nearest item

We can use a similar process to find the item that is closest to a given value. To do that, we will need to record
1. the index of the closest item, and
2. the distance between the closest item and our given value.

In [ ]:
def find_nearest(numbers, target):
    # index of the nearest
    i_n = 0
    
    # dist to the nearest
    d_n = abs(target - numbers[i_n])
    
    for i in range(1, len(numbers)):
        d = abs(target - numbers[i])
        if (d < d_n):
            d_n = d
            i_n = i
    
    # return the index, the value, and the distance to target
    return (i_n, numbers[i_n], d_n)

In [ ]:
target = 500
(i, n, d) = find_nearest(numbers, target)
print("The", i, "-th item", n, "is the nearest value to", target,", the difference is", d)

In [ ]:
target = 750
(i, n, d) = find_nearest(numbers, target)
print("The", i, "-th item", n, "is the nearest value to", target,", the difference is", d)

In [ ]:
target = 300
(i, n, d) = find_nearest(numbers, target)
print("The", i, "-th item", n, "is the nearest value to", target,", the difference is", d)

### Complexity

Our method works, but it is is not great because it runs very slow. For example, if we repeat our process 100 times, and see how many time it takes:

In [ ]:
import time

t0 = time.time()

for i in range(100):
    target = random.random() * 1000
    find_nearest(numbers, target)

t1 = time.time()

print("Calculation took", t1-t0, "seconds")

Our algorithm is slow, or we say it has a high **_time complexity_**, the number of calculation we need equals to $$m * n$$ where _m_ is the number of targets, and _n_ is the length of the list.

As a further reading: there is also a **_space complexity_**, which refers how much computer memory an algorithm needs. Normally, you cannot achieve both low _time complexity_ and low _space complexity_ at the same time, and there is always a trade-off between these two. And since it is relatively easier to increase the storage space of a computer than to make the CPU runs faster, _time complexity_ is usually more critical than _space complexity_.

The _time complex_ and _space complexity_ , are the two main parts of the domain [Computational Complexity](https://en.wikipedia.org/wiki/Computational_complexity)

### Finding the nearest item, faster

What if, we get ride of the obviously not clost enough values, and compare only those values that are close enough? That should save us a lot of time right?

But how?

Maybe we can round the values to intergers first?

Like we **first put all numbers to different drawers**.

In [ ]:
# a dictionary, representing the mapping between
# integers and list of numbers that can be rounded
# to those integers
# for example {0: [0.1, 0.3], 1:[1.5, 1.8], ..., 9:[9.2, 9.7]}

drawers = {}

for i in range(len(numbers)):
    n = numbers[i]  # i-th number
    n_int = int(n)  # rounded i-th number
    
    if n_int not in drawers:
        # create a new list using n_int as key,
        # and add (i, n) to the list
        drawers[n_int] = [(i, n)]
    else:
        # add (i, n) to the existing list
        drawers[n_int].append((i, n))

Then we can **first select the drawer** (compare the target with the rounded integers), and **then we compare numbers only from those drawers**

First we need a function that **compares our target with a drawer** (i.e., the list of tuples)

In [ ]:
# drawer is a list of (index, number) tuples
def find_nearest_drawer(drawer, target):
    # index of the nearest tuple
    i_n = 0
    
    # dist to the nearest tuple
    d_n = abs(target - drawer[i_n][1])
    
    for i in range(1, len(drawer)):
        d = abs(target - drawer[i][1])
        if (d < d_n):
            d_n = d
            i_n = i
    
    # remember drawer[i_n] is a tuple (index, number)
    # return the index, the value, and the distance to target
    return (drawer[i_n][0], drawer[i_n][1], d_n)

In [ ]:
def find_nearest_fast(drawers, target):
    drawer_keys = list(drawers)  # drawer keys are integers
    
    # we first find the nearest rounded integer to the target,
    # using the method we already written
    (i, n_int, d) = find_nearest(drawer_keys, target)
    
    # we can use the nearest integer (n_int) as the key,
    # and get the drawer (numbers that can be rounded to the key)
    drawer = drawers[n_int]
    
    # then we can just compare our target with the drawer
    (i, n, d) = find_nearest_drawer(drawer, target)
    
    # we should extend our range of search,
    # in case that the target is very close to the rounding boundary
    # for example, when target = 2.01,
    # it is also worth to compare it with values that are rounded to 1
    # when target = 2.99,
    # it is also worth to compare it with values that are rounded to 3
    
    if n_int - 1 in drawers:
        drawer_low = drawers[n_int - 1]
        (i_low, n_low, d_low) = find_nearest_drawer(drawer_low, target)

    if n_int + 1 in drawers:
        drawer_high = drawers[n_int + 1]
        (i_high, n_high, d_high) = find_nearest_drawer(drawer_high, target)
    
    # then we select the one with lowest distance to target
    if d_low < d:
        d = d_low
        i = i_low
        n = n_low
    
    if d_high < d:
        d = d_high
        i = i_high
        n = n_high
    
    # return the index, the value, and the closest distance to target
    return i, n, d

Lets check the results

In [ ]:
target = 500
(i, n, d) = find_nearest_fast(drawers, target)
print("The", i, "-th item", n, "is the nearest value to", target,", the difference is", d)

In [ ]:
target = 750
(i, n, d) = find_nearest_fast(drawers, target)
print("The", i, "-th item", n, "is the nearest value to", target,", the difference is", d)

In [ ]:
target = 300
(i, n, d) = find_nearest_fast(drawers, target)
print("The", i, "-th item", n, "is the nearest value to", target,", the difference is", d)

As well as the speed

In [ ]:
t0 = time.time()

for i in range(100):
    target = random.random() * 1000
    find_nearest_fast(drawers, target)

t1 = time.time()

print("Calculation took", t1-t0, "seconds")

By building us an integer drawer for floating numbers, we reduced the calculation time for almost **100 times!**

The problem we just solved, is called the [Nearest-neighbor](https://en.wikipedia.org/wiki/Nearest_neighbor_search), and the methods we used, the slower one is called [Linear Search](https://en.wikipedia.org/wiki/Nearest_neighbor_search#Linear_search), and the faster one is called [Space Partitioning](https://en.wikipedia.org/wiki/Nearest_neighbor_search#Space_partitioning)

## Experiencing, but try avoid implementing

The above example gave you an experience of how proper data structure and algorithm can significantly increase our problem-solving ability.

However, **when coding in python, we should try to avoid implement complex algorithms ourselves**.

This is because python itself is a scripting language which is already super slow. The advantage of python, is that it has access to a huge amount of libraries written in different other languages for solving different problems.

We should therefore always try to find proper libraries first, before we running out of options and having to do the work by ourselves.

## Typical Problems

### Sorting

Sorting is to sort a list of element in a particular order. For this task, Python <code>list</code> already gave us quite powerful implementation:

In [ ]:
a = [random.randint(0, 100) for i in range(10)]
a

In [ ]:
a.sort()
a

### Searching

For minium and maximum numbers, try to use the python <code>min</code> and <code>max</code> functions

In [ ]:
a = [random.randint(0, 100) for i in range(10)]

print(a)
print("the minimum is", min(a), ", the maximum is", max(a))

To obtain the index of the mininum or maximum item, we can use <code>numpy</code> library

In [ ]:
import numpy as np

a = [random.randint(0, 100) for i in range(10)]

print(a)
print("the index of the minimum is", np.argmin(a), ", the index of the maximum is", np.argmax(a))

To search for the nearest-neighbor, we can use the [sklearn library](https://scikit-learn.org/stable/modules/neighbors.html) 

### Shortest path

[Shortest path](https://en.wikipedia.org/wiki/Shortest_path_problem) is a type of problem we need to solve when, for example, we want to know how to travel as fast as possible from one location to another.

This type of problem is well-studied and there are many algorithms avaiable for it.

The most famous one is called [Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm). There is a [nice blog](https://medium.com/@azkardm/dijkstras-algorithm-in-python-finding-the-shortest-path-bcb3bcd4a4ea) explains it in detail.

### Computational geometry

[Computational geometry](https://en.wikipedia.org/wiki/Computational_geometry) represents a type of geometry-related problems that needs to be solved using algorithms, ranging from, for example, computing the intersection points of two curves, to computing the volume of a polyhedral.

This type of problems usually require a lot of math knowledge. For example, computing the [intersection of two lines](https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection) can be done by:

In [ ]:
# this code is taken from
# https://stackoverflow.com/questions/20677795/how-do-i-compute-the-intersection-point-of-two-lines
# 
# compute line-line intersection,
# where each line is specified by
# ((x1, y1), (x2, y2))

def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

In [ ]:
A = (0, 0)
B = (10, 0)
C = (5, -3)
D = (8, 7)

line_intersection((A, B), (C, D))

**Rhino 3D** is one of the most powerful library that we can use in Python for computational geometry!

You can not only code Python scripy inside Rhino (i.e., Rhino and Grasshopper script), also outside Rhino using [Rhino Inside](https://pypi.org/project/rhinoinside/) technology.